**Préambule**

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 20.7 MB/s 
     |████████████████████████████████| 73 kB 1.9 MB/s 
     |████████████████████████████████| 16.3 MB 57.1 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 54.7 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


**Q1:** 
  Les fichiers **IWSLT10_BTEC.train.en.txt** et **IWSLT10_BTEC.train.fr.txt** contiennent des données d'apprentissage. Le fichier **IWSLT10_BTEC.train.en.txt** contient des passage en anglais et le fichier **IWSLT10_BTEC.train.fr.txt** contient leur traduction en français en utilisant les commandes linux, ces fichiers servent de base pour les modèles et sont considérés comme la traduction "correcte" des fichiers.

**Q2:** Dans les fichiers tok.txt, les apostrophes sont représentées par "\&apos;", son code HTML. Cela est efféctué dans le but que les caractères spéciaux soient reconnus en tant que tel par notre programme. 

<span style="color:red">La difference importante est representé par les espaces entroduit entre chanque mot et la ponctuation. Le but de cette stratégie basique de tokenization est de réduire la taille du vocabulaire. Si on ne sépare pas les mots et la ponctuation on sera obligé à avoir plus de tokens dans le vocabularie. E.g. "Salut", "Salut!", "Salut,", "Salut?", etc. </span>

In [ ]:
%%shell
cd drive/MyDrive/tp_nmt/BTEC-en-fr
cd ./dev
awk -F '\' '{print $NF}' IWSLT10.devset1_CSTAR03.fr.txt > IWSLT10.devset1_CSTAR03.fr.clean.txt
awk -F '\' '{print $NF}' IWSLT10.devset1_CSTAR03.en.txt > IWSLT10.devset1_CSTAR03.en.clean.txt

cd ../test
awk -F '\' '{print $NF}' IWSLT09_BTEC.testset.fr.txt > IWSLT09_BTEC.testset.fr.clean.txt
awk -F '\' '{print $NF}' IWSLT09_BTEC.testset.en.txt > IWSLT09_BTEC.testset.en.clean.txt

cd ../train
awk -F '\' '{print $NF}' IWSLT10_BTEC.train.fr.txt > IWSLT10_BTEC.train.fr.clean.txt
awk -F '\' '{print $NF}' IWSLT10_BTEC.train.en.txt > IWSLT10_BTEC.train.en.clean.txt



In [ ]:
%%shell
cd drive/MyDrive/tp_nmt/
perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.clean.txt > ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt
perl tokenizer.perl -l en -lc < ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.clean.txt > ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt

perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.clean.txt > ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt
perl tokenizer.perl -l en -lc < ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.clean.txt > ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt

perl tokenizer.perl -l fr -lc < ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.clean.txt > ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.tok.txt
perl tokenizer.perl -l en -lc < ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.clean.txt > ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.tok.txt


Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1


**Q3:** Étant donné un ensemble de données, il faut utiliser cette ressource pour l'apprentissge, le réglage des paramètres et le test de la précision du modèle. Nous ne pouvons pas effectuer cela avec les mêmes données. Nous choisissons ainsi un corpus séparé en trois parties (train, dev et test).

Les fichiers train ont pour but d'entraîner le classificateur. Les fichiers dev, utilisés pendant l'évaluation du classificateur avec différentes configurations ou variations dans la représentation des caractéristiques. <span style="color:red">Je ne comprends pas ce que vous voulez dire. En tout cas le dev et utilisé pour évaluer le modèle pendent l'entrainement est décidé s'il faut l'arreter, ou encore pour établir la meilleure configuration de hyperparamétres. </span>  Les fichiers test contiennent les données sur lequel on vérifie finalement la précision du classificateur et on obtient des résultats non biaisés.

**Q4:** Le but du vocabulaire est de permettre à notre programme de reconnaitre les mots qui font référence à la même chose (exemple: reservations, reservation). <span style="color:red"> C'est à dire? Le vocabulaire permet d'associer à chaque mot un idéntifiant unique et un vecteur qui le represente et dont les valuers seronts appris par le réseau à travers l'entrainement. </span>

On utilise un vocabulaire issu de la phase d'apprentissage pour des raisons d'optimisation. Le programme n'a que des mots qu'il va utiliser. 

In [ ]:
%%shell
cd drive/MyDrive/tp_nmt/
onmt_build_vocab -config config-base.yaml -n_sample -1

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 18:20:16,242 INFO] Counter vocab from -1 samples.
[2022-02-27 18:20:16,242 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-27 18:20:16,258 INFO] train's transforms: TransformPipe()
[2022-02-27 18:20:16,662 INFO] Counters src:9978
[2022-02-27 18:20:16,663 INFO] Counters tgt:8194


**Q5:** Counters src:9978 fait référence au nombre de mots dans le vocabulaire source.

Counters tgt:8194 fait référence au nombre de mots dans le vocabulaire du ficher de sortie (ou target).



In [ ]:
%%shell
cd drive/MyDrive/tp_nmt/
onmt_train -config ./config-base.yaml

[2022-02-27 18:20:36,761 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 18:20:36,931 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 18:20:36,931 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 18:20:36,932 INFO] Parsed 2 corpora from -data.
[2022-02-27 18:20:36,933 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 18:20:36,933 INFO] Loading vocab from text file...
[2022-02-27 18:20:36,933 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 18:20:36,959 INFO] Loaded src vocab has 9978 tokens.
[2022-02-27 18:20:36,964 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 18:20:36,990 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 18:20:36,994 INFO] Building fields with vocab in counters...
[2022-02-27 18:20:37,004 INFO]  * tgt vocab size: 8198.
[2022-02-27 18:20:37,019 INFO]  * src vocab size: 

**Q6:** La valeur de acc fait référence au niveau de précision du modèle. Plus cette valeur est élevée, plus c'est un bon indicateur de notre modèle. La valeur de ppl est la pérplexité. Plus elle est faible, plus c'est bon résultat pour notre modèle.  <span style="color:red"> Définition mathématique de acc et ppl? Intuition dérrière la définition de perplexity? </span>

Q7:
>valid_steps : effectue une validation tous les X pas.

>train_steps : le nombre d'étapes d'apprentissage. <span style="color:red">C'est quoi un pas d'apprentissage? (vous n'avez que traduit le nom de la variable sans expliquer sa signification). train_step = nombre total d'étapes d'optimisation des poids du modèle, où chaque étape est constituée d'un passage en avant d'un lot (batch) de données d'apprentissage dans le modèle (forward pass) et d'un passage en arrière des gradients générés par la fonction de perte (backward pass), suivi par une mise à jours des poids du modèle. </span>

>enc_layers: le nombre de couches de l'encodeur.  

>dec_layers: le nombre de couches du décodeur.  

> enc_rnn_size:  la taille des états cachéq de l'encodeur RNN.

> dec_rnn_size: la taille des états cachéq du décodeur RNN. 

> Batch_size: le nombre d'échantillons qui seront propagés dans le réseau. <span style="color:red"> ...dans la même étape d'entrainement/évalutaion (c'est un précisation importante) </span>

**Q8:** Le score BLEU est compris entre 0 et 1, ici il est affiché en pourcentage. Plus la valeur est élevée, plus les textes sont similaires. <span style="color:red"> Similaires à quoi? Est-ce qu'un score BLEU plus élévé veut toujours dire que la trad est méilleure ? -> non</span>

**Q9:**

In [58]:
%%shell
cd drive/MyDrive/tp_nmt/models/base
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred625.txt
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred1250.txt
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred1875.txt
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred2000.txt


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 23:53:59,741 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 23:54:03,637 INFO] PRED AVG SCORE: -1.2845, PRED PPL: 3.6129
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

In [59]:
%%shell
cd drive/MyDrive/tp_nmt/
perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/base/pred2000.txt


BLEU = 9.54, 47.0/17.7/9.3/3.2 (BP=0.762, ratio=0.786, hyp_len=2810, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 12.31, 41.0/16.4/9.2/3.7 (BP=1.000, ratio=1.113, hyp_len=3978, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 18.17, 52.8/24.7/15.1/7.4 (BP=0.931, ratio=0.933, hyp_

**Q9:** Le modèle pred2000 à le meilleur score.

**Q10:** On obtient un meilleur score BLEU en augmentant le nombre de train_steps puisque cela permet plus de précision. <span style="color:red"> (oui mais, pourquoi?)</span>  On augmente le train_steps jusqu'à ce que le score BLEU converge. L'option -early_stopping  **X** permet d'arreter l'apprentissage quand le modèle ne montre pas d'amélioration **X** fois de suite. Il est donc possible de donner un très grand nombre de train_steps et de donner un condition avec early_stopping pour obtenir le nobre optimal de train_steps.

Ici, on va essayer avec 6000 train-steps. J'ai essayé de faire la traduction et l'obtention du score bleu dans le même bloc mais je n'ai pas pris en compte les models qui sont produits tous les 625 steps et donc il y a une erreur, mais l'entrainnement est fini, dans le bloc suivant je fais la traduction et je calcule le score bleu.

In [53]:
%%shell
cd drive/MyDrive/tp_nmt/
onmt_train -config ./config-6000step.yaml -early_stopping 2
cd models/6000step
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2000.txt
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/6000step/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/6000step/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/6000step/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/6000step/pred2000.txt

[2022-02-27 22:50:02,104 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 22:50:02,106 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 22:50:02,106 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 22:50:02,107 INFO] Parsed 2 corpora from -data.
[2022-02-27 22:50:02,108 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 22:50:02,108 INFO] Loading vocab from text file...
[2022-02-27 22:50:02,108 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 22:50:02,135 INFO] Loaded src vocab has 9978 tokens.
[2022-02-27 22:50:02,141 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 22:50:02,161 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 22:50:02,166 INFO] Building fields with vocab in counters...
[2022-02-27 22:50:02,176 INFO]  * tgt vocab size: 8198.
[2022-02-27 22:50:02,190 INFO]  * src vocab size: 

CalledProcessError: ignored

In [54]:
%%shell
cd drive/MyDrive/tp_nmt/models/6000step
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt
onmt_translate -model model_step_2500.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2500.txt
onmt_translate -model model_step_3125.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred3125.txt
onmt_translate -model model_step_3750.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred3750.txt
onmt_translate -model model_step_4375.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred4375.txt
onmt_translate -model model_step_5000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred5000.txt
onmt_translate -model model_step_5625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred5625.txt
onmt_translate -model model_step_6000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred6000.txt
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/6000step/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/6000step/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/6000step/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/6000step/pred2500.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/6000step/pred3125.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/6000step/pred3750.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/6000step/pred4375.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/6000step/pred5000.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/6000step/pred5625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/6000step/pred6000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 23:24:45,551 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 23:24:49,658 INFO] PRED AVG SCORE: -1.1664, PRED PPL: 3.2104
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

Le score bleu augmente avec le nombre de train_steps, mais l'execution du training prends énormement de temps, on a donc décider de ne pas augmenter plus le nombre maximal de train_steps.

Q11: Nous n'obtenons pas de meilleurs résultats avec plus de couche 

<span style="color:red">Pourquoi? en tout cas, il aurait fallu utiliser l'option early_stopping pour bien repondre à cette question (ainsi que les deux prochaines). En effet, si l'on utilise pas cette option, on risque de comparer des modèles qui sont sous-entrainé (c'est votre cas) ou sur-entrainés (overfitted),  sans donc pouvoir en tirer des conclusions. En effet, le nombre d'étapes d'entrainement nécessaires à l'optimization du modèle varie selon la quantité de paramétres (poids) à optimizer, ce qui est le cas lorsqu'on varie le nombre de couches ou leur taille.  </span>

In [ ]:
%%shell
cd drive/MyDrive/tp_nmt/
onmt_train -config ./config-2couche.yaml


[2022-02-27 18:57:50,687 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 18:57:50,689 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 18:57:50,690 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 18:57:50,690 INFO] Parsed 2 corpora from -data.
[2022-02-27 18:57:50,691 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 18:57:50,691 INFO] Loading vocab from text file...
[2022-02-27 18:57:50,692 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 18:57:50,718 INFO] Loaded src vocab has 9978 tokens.
[2022-02-27 18:57:50,723 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 18:57:50,743 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 18:57:50,748 INFO] Building fields with vocab in counters...
[2022-02-27 18:57:50,759 INFO]  * tgt vocab size: 8198.
[2022-02-27 18:57:50,774 INFO]  * src vocab size: 

In [44]:
%%shell
cd drive/MyDrive/tp_nmt/models/2couches
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2000.txt

[2022-02-27 21:50:13,043 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 21:50:17,764 INFO] PRED AVG SCORE: -1.2682, PRED PPL: 3.5543
[2022-02-27 21:50:20,045 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current b

In [45]:
%%shell
cd drive/MyDrive/tp_nmt/
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/2couches/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/2couches/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/2couches/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/2couches/pred2000.txt

BLEU = 7.49, 44.0/14.1/7.9/1.9 (BP=0.767, ratio=0.790, hyp_len=3010, ref_len=3808)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 11.65, 48.9/19.0/10.6/4.3 (BP=0.811, ratio=0.827, hyp_len=3150, ref_len=3808)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 13.79, 49.3/20.0/11.0/4.1 (BP=0.954, ratio=0.955, hyp

In [46]:
%%shell
cd drive/MyDrive/tp_nmt/
onmt_train -config ./config-3couche.yaml
cd models/3couches
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2000.txt
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/3couches/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/3couches/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/3couches/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/3couches/pred2000.txt


[2022-02-27 21:52:48,477 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 21:52:48,482 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 21:52:48,482 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 21:52:48,483 INFO] Parsed 2 corpora from -data.
[2022-02-27 21:52:48,484 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 21:52:48,484 INFO] Loading vocab from text file...
[2022-02-27 21:52:48,484 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 21:52:48,515 INFO] Loaded src vocab has 9978 tokens.
[2022-02-27 21:52:48,520 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 21:52:48,541 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 21:52:48,545 INFO] Building fields with vocab in counters...
[2022-02-27 21:52:48,557 INFO]  * tgt vocab size: 8198.
[2022-02-27 21:52:48,573 INFO]  * src vocab size: 

**Q12:** En augmentant le nombre d'unités pour l'encodeur et le décodeur, on obtient de meilleurs résultats. Ces résultats varient d'une execution à l'autre mais on remarqué de meilleurs résultats pour 384 unités. <span style="color:red"> Pourquoi? </span>


In [51]:
%%shell
cd drive/MyDrive/tp_nmt/
onmt_train -config ./config-384unit.yaml
cd models/384unit
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2000.txt
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/384unit/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/384unit/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/384unit/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/384unit/pred2000.txt

[2022-02-27 22:12:33,981 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 22:12:33,983 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 22:12:33,983 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 22:12:33,984 INFO] Parsed 2 corpora from -data.
[2022-02-27 22:12:33,984 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 22:12:33,984 INFO] Loading vocab from text file...
[2022-02-27 22:12:33,984 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 22:12:34,009 INFO] Loaded src vocab has 9978 tokens.
[2022-02-27 22:12:34,015 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 22:12:34,035 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 22:12:34,039 INFO] Building fields with vocab in counters...
[2022-02-27 22:12:34,050 INFO]  * tgt vocab size: 8198.
[2022-02-27 22:12:34,065 INFO]  * src vocab size: 

In [52]:
%%shell
cd drive/MyDrive/tp_nmt/
onmt_train -config ./config-512unit.yaml
cd models/512unit
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2000.txt
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/512unit/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/512unit/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/512unit/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/512unit/pred2000.txt

[2022-02-27 22:29:13,578 INFO] Missing transforms field for train data, set to default: [].
[2022-02-27 22:29:13,581 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-27 22:29:13,581 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-27 22:29:13,582 INFO] Parsed 2 corpora from -data.
[2022-02-27 22:29:13,583 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-27 22:29:13,583 INFO] Loading vocab from text file...
[2022-02-27 22:29:13,583 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-27 22:29:13,615 INFO] Loaded src vocab has 9978 tokens.
[2022-02-27 22:29:13,621 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-27 22:29:13,642 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-27 22:29:13,646 INFO] Building fields with vocab in counters...
[2022-02-27 22:29:13,657 INFO]  * tgt vocab size: 8198.
[2022-02-27 22:29:13,672 INFO]  * src vocab size: 

**Q13:** On n'observe pas d'amélioration de résultats en ajoutant un méchaniqme d'attention. <span style="color:red"> où sont les logs? Vous n'observé pas d'amélioration car votre modèle est sous entrainé (vous entrainé pour 2000 steps seulements). En effet, en rajoutant le mechanisme d'attention on rajoute des paramètres au modèle, ce qui implique que notre modèle aura besoin de plus de temps d'entrainement. </span>


In [ ]:
%%shell
cd drive/MyDrive/tp_nmt/
onmt_train -config ./config-mlp.yaml
cd models/mlp
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2000.txt
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/mlp/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/mlp/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/mlp/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/mlp/pred2000.txt

**Q14:** Le beam search est un algorithme de recherche heuristique qui explore un graphe en ne considérant qu'un ensemble limité de fils de chaque nœud. On observe une amélioration des résultats quand on augmente le beam_size. Par défault, le beam_size est de 5.

<span style="color:red"> C'est quoi les noeuds et les fils dans notre cas ? --> les tokens de chaque étape de décodage de la phrase cible, c.à.d. de la traduction </span>


**Q15:** On obtiens des résultats inférieurs à la base en fixant le beam_size à 1 et de meilleurs résultats quand on le fixe à 10. <span style="color:red"> Pourquoi? </span>


In [49]:
%%shell
cd drive/MyDrive/tp_nmt/
cd models/base
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt -beam_size 1
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt -beam_size 1
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt -beam_size 1
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2000.txt -beam_size 1
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/base/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/base/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/base/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/base/pred2000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 22:10:17,879 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 22:10:19,442 INFO] PRED AVG SCORE: -1.5322, PRED PPL: 4.6283
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

In [50]:
%%shell
cd drive/MyDrive/tp_nmt/
cd models/base
onmt_translate -model model_step_625.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred625.txt -beam_size 10
onmt_translate -model model_step_1250.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1250.txt -beam_size 10
onmt_translate -model model_step_1875.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred1875.txt -beam_size 10
onmt_translate -model model_step_2000.pt -src ../../BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output pred2000.txt -beam_size 10
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/base/pred625.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/base/pred1250.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < models/base/pred1875.txt
perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt< models/base/pred2000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 22:11:11,119 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 22:11:18,490 INFO] PRED AVG SCORE: -1.3008, PRED PPL: 3.6724
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

**Q16:** Le meilleur score BLEU obtenu est de 31.76, pour le modèle avec 6000 train_steps. On va maintenant tester ce modèle sur les testsets.

In [57]:
%%shell
cd drive/MyDrive/tp_nmt/models/6000step
onmt_translate -model model_step_6000.pt -src ../../BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output pred6000.txt
cd ../..
perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < models/6000step/pred6000.txt


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 23:49:57,643 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 23:50:02,189 INFO] PRED AVG SCORE: -0.6034, PRED PPL: 1.8283
BLEU = 31.61, 64.6/39.6/27.8/18.9 (BP=0.929, ratio=0.931, hyp_len=3329, ref_len=3574)
It is 

On obtient un score BLEU de 31.61, assez proche dur score obtenu lors de la phase dev.

**Q17:** La qualité de la traduction me semble être meilleure pour le meilleur modele, par exemple la première phrase "Help" à été interprété par le premier modèle comme "At it", ce qui n'à pas trop de sense, et par le deuxième modèle par "Stop thief", toujour pas exactement correct mais ça a plus de sens. Cependant d'autres phrases comme "Could you drive more slowly, please?" on été interprété par le premier modèle comme "Can you take me, please ?" et par le deuxième comme "Can you speak more slowly, please ?". Ici le sens est plus présent pour le premier modèle mais la grammaire est maladroite, alors que le deuxième modèle à fait un erreur sur un seul mot.